In [1]:
### Utilities
import os
import json
import numpy as np
from time import time
from tqdm import tqdm
import copy

### Torch
import torch
from torch import nn
from torch.optim import AdamW
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torch.optim.lr_scheduler import MultiStepLR

### Quantization
from torch.quantization.qconfig import QConfig
from torch.quantization.observer import MinMaxObserver

### Custom
from src.model import ResNet20
from src.train import validate
from src.utils import Accuracy, get_model_size
from src.qmodel import *

### Configuration

In [2]:
with open("cfg/64_200.json") as configurations:
    cfg, cfg_CIFAR, cfg_dataloader_train, cfg_dataloader_test, cfg_train = json.load(configurations).values()

### Data

In [3]:
trainset = CIFAR10(transform=transforms.Compose([
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomCrop(32, 4),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])]
                    ), **cfg_CIFAR, train=True)
trainloader = DataLoader(trainset, **cfg_dataloader_train)

testset = CIFAR10(transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])]
                    ), **cfg_CIFAR, train=False)
testloader = DataLoader(testset, **cfg_dataloader_test)

n_q, idxes = 10, torch.randperm(len(trainset))
qloader = DataLoader([trainset[idxes[i]] for i in range(n_q)], **cfg_dataloader_train)

Files already downloaded and verified
Files already downloaded and verified


### Model

In [4]:
is_cuda = torch.cuda.is_available()
from_checkpoint = os.path.exists(cfg["checkpoint_path"])
cfg["device"] = torch.device("cuda") if is_cuda \
            else torch.device("cpu")
cfg["q_device"] = torch.device("cpu")

ResNet = ResNet20().to(cfg["device"])
CELoss = nn.CrossEntropyLoss(reduction="sum")
Acc = Accuracy(reduction="sum")

if from_checkpoint:
    checkpoint = torch.load(cfg["checkpoint_path"], map_location=cfg["device"])
    ResNet.load_state_dict(checkpoint["state_dict"])


In [5]:
print("Load Checkpoint")
ce, acc = validate(testloader, ResNet, CELoss, Acc, cfg["device"], verbose=True)
print("Cross Entropy: {:.3f}, Accuracy: {:.3f}".format(ce.avg, acc.avg))
_ = ResNet.to(cfg["q_device"])

Load Checkpoint


100%|██████████| 10/10 [00:47<00:00,  4.79s/it]

Cross Entropy: 0.267, Accuracy: 0.921


In [14]:
qconfig_int2 = QConfig(
    activation=MinMaxObserver.with_args(
        dtype=torch.quint8,
        quant_min=0,
        quant_max=3
    ),
    weight=MinMaxObserver.with_args(
        dtype=torch.qint8,
        quant_min=-2,
        quant_max=1
    )
)
    
ResNetPTQint2 = torch.quantization.QuantWrapper(
    copy.deepcopy(ResNet)
)

ResNetPTQint2.qconfig = qconfig_int2
torch.quantization.prepare(ResNetPTQint2, inplace=True)

print("Compute Statistics")
ce, acc = validate(qloader, ResNetPTQint2, CELoss, Acc, cfg["q_device"], verbose=True)
_ = torch.quantization.convert(ResNetPTQint2, inplace=True)

print("Evaluate")
ce, acc = validate(testloader, ResNetPTQint2, CELoss, Acc, cfg["q_device"], verbose=True)
print("Cross Entropy: {:.3f}, Accuracy: {:.3f}".format(ce.avg, acc.avg))

torch.save(ResNetPTQint2.state_dict(), cfg["checkpoint_path"]+"_int2")

Compute Statistics


100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


Evaluate


100%|██████████| 10/10 [00:23<00:00,  2.35s/it]

Cross Entropy: 20.223, Accuracy: 0.111


In [15]:
qconfig_int4 = torch.quantization.qconfig.QConfig(
    activation=MinMaxObserver.with_args(
        dtype=torch.quint8,
        quant_min=0,
        quant_max=15
    ),
    weight=MinMaxObserver.with_args(
        dtype=torch.qint8,
        quant_min =-8,
        quant_max =7,
    )
)

ResNetPTQint4 = torch.quantization.QuantWrapper(
    copy.deepcopy(ResNet)
)

ResNetPTQint4.qconfig = qconfig_int4
torch.quantization.prepare(ResNetPTQint4, inplace=True)

print("Compute Statistics")
ce, acc = validate(qloader, ResNetPTQint4, CELoss, Acc, cfg["q_device"], verbose=True)
_ = torch.quantization.convert(ResNetPTQint4, inplace=True)
print("Evaluate")
ce, acc = validate(testloader, ResNetPTQint4, CELoss, Acc, cfg["q_device"], verbose=True)
print("Cross Entropy: {:.3f}, Accuracy: {:.3f}".format(ce.avg, acc.avg))

torch.save(ResNetPTQint4.state_dict(), cfg["checkpoint_path"]+"_int4")

Compute Statistics


100%|██████████| 1/1 [00:00<00:00,  1.30it/s]


Evaluate


100%|██████████| 10/10 [00:22<00:00,  2.30s/it]

Cross Entropy: 8.848, Accuracy: 0.296


In [16]:
qconfig_int8 = torch.quantization.qconfig.QConfig(
    activation=MinMaxObserver.with_args(
        dtype=torch.quint8
    ),
    weight=MinMaxObserver.with_args(
        dtype=torch.qint8
    )
)

ResNetPTQint8 = torch.quantization.QuantWrapper(
    copy.deepcopy(ResNet)
)

ResNetPTQint8.qconfig = qconfig_int8
torch.quantization.prepare(ResNetPTQint8, inplace=True)

print("Compute Statistics")
ResNetPTQint8.eval()
ce, acc = validate(qloader, ResNetPTQint8, CELoss, Acc, cfg["q_device"], verbose=True)
_ = torch.quantization.convert(ResNetPTQint8, inplace=True)
ResNetPTQint8.eval()
print("Evaluate")
ce, acc = validate(testloader, ResNetPTQint8, CELoss, Acc, cfg["q_device"], verbose=True)
print("Cross Entropy: {:.3f}, Accuracy: {:.3f}".format(ce.avg, acc.avg))

torch.save(ResNetPTQint8.state_dict(), cfg["checkpoint_path"]+"_int8")

Compute Statistics


100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


Evaluate


100%|██████████| 10/10 [00:23<00:00,  2.30s/it]

Cross Entropy: 0.411, Accuracy: 0.881


In [17]:
qconfig_int8 = torch.quantization.qconfig.QConfig(
    activation=MinMaxObserver.with_args(
        dtype=torch.quint8
    ),
    weight=MinMaxObserver.with_args(
        dtype=torch.qint8
    )
)

ResNetPTQFuseint8 = copy.deepcopy(ResNet)

modules_to_fuse = [
['0','1', '2'],

['3.f.0', '3.f.1','3.f.2'], ['3.f.3', '3.f.4',],
['4.f.0', '4.f.1','4.f.2'], ['4.f.3', '4.f.4',],
['5.f.0', '5.f.1','5.f.2'], ['5.f.3', '5.f.4',],

['6.f.0', '6.f.1','6.f.2'], ['6.f.3', '6.f.4',],
['6.c.0','6.c.1'],

['7.f.0', '7.f.1','7.f.2'], ['7.f.3', '7.f.4',],
['8.f.0', '8.f.1','8.f.2'], ['8.f.3', '8.f.4',],
['9.f.0', '9.f.1','9.f.2'], ['9.f.3', '9.f.4',],

['10.f.0', '10.f.1','10.f.2'], ['10.f.3', '10.f.4',],
['10.c.0','10.c.1'],

['11.f.0', '11.f.1','11.f.2'], ['11.f.3', '11.f.4',],
['12.f.0', '12.f.1','12.f.2'], ['12.f.3', '12.f.4',],
['13.f.0', '13.f.1','13.f.2'], ['13.f.3', '13.f.4',]
]

torch.quantization.fuse_modules(ResNetPTQFuseint8, modules_to_fuse, inplace=True)
ResNetPTQFuseint8 = torch.quantization.QuantWrapper(
    ResNetPTQFuseint8
)

ResNetPTQFuseint8.qconfig = qconfig_int8
torch.quantization.prepare(ResNetPTQFuseint8, inplace=True)

print("Compute Statistics")
ResNetPTQFuseint8.eval()
ce, acc = validate(qloader, ResNetPTQFuseint8, CELoss, Acc, cfg["q_device"], verbose=True)
_ = torch.quantization.convert(ResNetPTQFuseint8, inplace=True)
ResNetPTQFuseint8.eval()
print("Evaluate")
ce, acc = validate(testloader, ResNetPTQFuseint8, CELoss, Acc, cfg["q_device"], verbose=True)
print("Cross Entropy: {:.3f}, Accuracy: {:.3f}".format(ce.avg, acc.avg))

torch.save(ResNetPTQFuseint8.state_dict(), cfg["checkpoint_path"]+"_int8")

Compute Statistics


100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


Evaluate


100%|██████████| 10/10 [00:15<00:00,  1.60s/it]

Cross Entropy: 0.290, Accuracy: 0.913


In [18]:
QuantizedNN = quantize_merge_model(ResNet)
print("Evaluate")
ce, acc = validate(qloader, QuantizedNN, CELoss, Acc, cfg["q_device"], verbose=True)
QuantizedNN.apply(compile_module)
ce, acc = validate(testloader, QuantizedNN, CELoss, Acc, cfg["q_device"], verbose=True)
print("Cross Entropy: {:.3f}, Accuracy: {:.3f}".format(ce.avg, acc.avg))

Evaluate


100%|██████████| 10/10 [00:16<00:00,  1.65s/it]

Cross Entropy: 0.288, Accuracy: 0.914


#### Compare size and speed execution

In [19]:
print("--------------------------------------------------------------------")
print("---------------------------Original Model---------------------------")
print("--------------------------------------------------------------------")
print("Size: {:.3f} MB".format(get_model_size(ResNet)))
time_norm = []
for i in (pbar := tqdm(range(5))):
    start_time = time()
    validate(testloader, ResNet, CELoss, Acc, cfg["q_device"])
    time_norm.append(time() - start_time)
    pbar.set_description('Time {:.3f}s \u00B1 {:.3f}s'.format(np.mean(time_norm), np.std(time_norm)))

print("--------------------------------------------------------------------")
print("-----------------------Pytorch Quantized Model----------------------")
print("--------------------------------------------------------------------")
print("Size: {:.3f} MB".format(get_model_size(ResNetPTQint8)))
time_quant = []
for i in (pbar := tqdm(range(5))):
    start_time = time()
    validate(testloader, ResNetPTQint8, CELoss, Acc, cfg["q_device"])
    time_quant.append(time() - start_time)
    pbar.set_description('Time {:.3f}s \u00B1 {:.3f}s'.format(np.mean(time_quant), np.std(time_quant)))

print("--------------------------------------------------------------------")
print("------------------Pytorch Fused Quantized Model---------------------")
print("--------------------------------------------------------------------")
print("Size: {:.3f} MB".format(get_model_size(ResNetPTQFuseint8)))
time_quant = []
for i in (pbar := tqdm(range(5))):
    start_time = time()
    validate(testloader, ResNetPTQFuseint8, CELoss, Acc, cfg["q_device"])
    time_quant.append(time() - start_time)
    pbar.set_description('Time {:.3f}s \u00B1 {:.3f}s'.format(np.mean(time_quant), np.std(time_quant)))

print("--------------------------------------------------------------------")
print("---------------------Hands-on Quantized Model-----------------------")
print("--------------------------------------------------------------------")
print("Size: {:.3f} MB".format(get_model_size(QuantizedNN)))
time_quant = []
for i in (pbar := tqdm(range(5))):
    start_time = time()
    validate(testloader, QuantizedNN, CELoss, Acc, cfg["q_device"])
    time_quant.append(time() - start_time)
    pbar.set_description('Time {:.3f}s \u00B1 {:.3f}s'.format(np.mean(time_quant), np.std(time_quant)))

--------------------------------------------------------------------
---------------------------Original Model---------------------------
--------------------------------------------------------------------
Size: 1.596 MB


Time 39.570s ± 0.862s: 100%|██████████| 5/5 [03:17<00:00, 39.57s/it]


--------------------------------------------------------------------
-----------------------Pytorch Quantized Model----------------------
--------------------------------------------------------------------
Size: 0.485 MB


Time 22.974s ± 0.332s: 100%|██████████| 5/5 [01:54<00:00, 22.98s/it]


--------------------------------------------------------------------
------------------Pytorch Fused Quantized Model---------------------
--------------------------------------------------------------------
Size: 0.430 MB


Time 15.628s ± 0.179s: 100%|██████████| 5/5 [01:18<00:00, 15.63s/it]


--------------------------------------------------------------------
---------------------Hands-on Quantized Model-----------------------
--------------------------------------------------------------------
Size: 0.425 MB


Time 16.834s ± 0.339s: 100%|██████████| 5/5 [01:24<00:00, 16.84s/it]
